In [1]:
import numpy as np

In [2]:
cd insight_testsuite/tests/test_1/input/

/notebooks/find-political-donors_beta/insight_testsuite/tests/test_1/input


In [3]:
text_file = open('itcont.txt')
itcont_list = text_file.read().split('\n')

In [4]:
len(itcont_list)

8

In [5]:
itcont_list

['C00629618|N|TER|P|201701230300133512|15C|IND|PEREZ, JOHN A|LOS ANGELES|CA|90017|PRINCIPAL|DOUBLE NICKEL ADVISORS|01032017|40|H6CA34245|SA01251735122|1141239|||2012520171368850783',
 'C00177436|N|M2|P|201702039042410894|15|IND|DEEHAN, WILLIAM N|ALPHARETTA|GA|300047357|UNUM|SVP, SALES, CL|01312017|384||PR2283873845050|1147350||P/R DEDUCTION ($192.00 BI-WEEKLY)|4020820171370029337',
 'C00384818|N|M2|P|201702039042412112|15|IND|ABBOTT, JOSEPH|WOONSOCKET|RI|028956146|CVS HEALTH|VP, RETAIL PHARMACY OPS|01122017|250||2017020211435-887|1147467|||4020820171370030285',
 'C00177436|N|M2|P|201702039042410893|15|IND|SABOURIN, JAMES|LOOKOUT MOUNTAIN|GA|307502818|UNUM|SVP, CORPORATE COMMUNICATIONS|01312017|230||PR1890575345050|1147350||P/R DEDUCTION ($115.00 BI-WEEKLY)|4020820171370029335',
 'C00177436|N|M2|P|201702039042410895|15|IND|JEROME, CHRISTOPHER|FALMOUTH|ME|041051896|UNUM|EVP, GLOBAL SERVICES|01312017|384||PR2283905245050|1147350||P/R DEDUCTION ($192.00 BI-WEEKLY)|4020820171370029342',
 'C

In [6]:
# Provide several solutions based on potential complication in the data
# Location based: Given the information that the incoming data will always follow the FEC format, we will used | splitter and location
# To prepare the data
def info_extract(FEC_list):
    """
    Read items delimited by '|' for each entry and keep only the core info
    --------------
    input: 
    FEC_list: list of strings, representing one single entry from the input file,
    output: 
    key_info: length 5 dict 
    or empty
    """
    # Check if length of the list is correct:
    if len(FEC_list) < 21:
        return

    # Check validity of Other_ID, CMTE_ID, TRANSACTION_DT and TRANSACTION_AMT
    if (len(FEC_list[15]) == 0) & (len(FEC_list[0]) > 0) & (len(FEC_list[13]) > 0) & (len(FEC_list[14]) > 0):
        key_info = {}
        key_info['CMTE_ID'] = FEC_list[0]
        key_info['ZIP_CODE'] = FEC_list[10][:5]
        key_info['TRANSACTION_DT'] = FEC_list[13]
        key_info['TRANSACTION_AMT'] = FEC_list[14]
        key_info['OTHER_ID'] = FEC_list[15]
        return key_info
    else:
        return

In [10]:
# Visualize the current splitting results
itcont = '|'.join(itcont_list).split('|')
itcont = itcont[:-1]    # Remove the space at the end, the final length of list should be n * 21
i=0
while (i+1)*21 < len(itcont)+1:
    #print(itcont[i*21: (i+1)*21])
    print(info_extract(itcont[i*21: (i+1)*21]))
    i+=1

None
{'OTHER_ID': '', 'TRANSACTION_AMT': '384', 'ZIP_CODE': '30004', 'CMTE_ID': 'C00177436', 'TRANSACTION_DT': '01312017'}
{'OTHER_ID': '', 'TRANSACTION_AMT': '250', 'ZIP_CODE': '02895', 'CMTE_ID': 'C00384818', 'TRANSACTION_DT': '01122017'}
{'OTHER_ID': '', 'TRANSACTION_AMT': '230', 'ZIP_CODE': '30750', 'CMTE_ID': 'C00177436', 'TRANSACTION_DT': '01312017'}
{'OTHER_ID': '', 'TRANSACTION_AMT': '384', 'ZIP_CODE': '04105', 'CMTE_ID': 'C00177436', 'TRANSACTION_DT': '01312017'}
{'OTHER_ID': '', 'TRANSACTION_AMT': '333', 'ZIP_CODE': '02895', 'CMTE_ID': 'C00384818', 'TRANSACTION_DT': '01122017'}
{'OTHER_ID': '', 'TRANSACTION_AMT': '384', 'ZIP_CODE': '04105', 'CMTE_ID': 'C00177436', 'TRANSACTION_DT': '01312017'}


In [8]:
def median_zip_old(itcont_list):
    """
    input:
        itcont_list: list, donation information entries
    output:
        output_by_zip: list of strings: 'recipient|zip|running_median|donation_number|total_donation'
    """
    input_len = len(itcont_list)
    dict_zip = {}
    output_by_zip = []

    for i in range(input_len):  # Perform serial processing for each entry
        itcont = itcont_list[i].split('|')
        core_info = info_extract(itcont)
        if core_info is None:   # Check if the entry contains core information
            continue

        # Check if ZIP and tranaction amount is valid
        try:
            len_check = core_info['ZIP_CODE'][4]  # check if 'ZIP_CODE' has at least 5 characters
            zip_code = int(core_info['ZIP_CODE'])  # check if 'ZIP_CODE' contains only digit
            trans_amt = int(core_info['TRANSACTION_AMT'])  # check if 'TRANSACTION_AMT' contains only digit
        except:
            continue

        # Create one dictionary for each recipient and zip
        key = core_info['CMTE_ID'] + core_info['ZIP_CODE']
        # Update the dictionary with new transaction amount
        if key not in dict_zip:       # First entry, initiate the list
            list_zip = []
        else:
            list_zip = dict_zip[key]

        # Obtain the running median given the current information and new transaction_amount
        print("trans_amt={a}, list_zip={b}".format(a=trans_amt, b=list_zip))

In [65]:
# Validate the information of key entry
median_zip_old(itcont_list)

trans_amt=384, list_zip=[]
trans_amt=250, list_zip=[]
trans_amt=230, list_zip=[]
trans_amt=384, list_zip=[]
trans_amt=333, list_zip=[]
trans_amt=384, list_zip=[]


In [ ]:
def pop_in_arr_below(count_arr_below, value):
    """
    input:
        count_arr_below: list of integers, old below median heap
        value: int, value to be popped into the below median heap
    output:
        count_arr_below: list of integers, new below median heap
    """
    if value <= len(count_arr_below) - 1:
        count_arr_below[value] += 1
    else:
        count_arr_below_max = len(count_arr_below) - 1
        count_arr_below_new = [0] * (value + 1)
        count_arr_below_new[:count_arr_below_max + 1] = count_arr_below
        count_arr_below = count_arr_below_new
        count_arr_below[len(count_arr_below) - 1] += 1

    return count_arr_below

In [ ]:
# Test pop in array below
value = np.array([4, 3, 1, 0])
for i in range(len(value)):
    count_arr_ori = np.array([1,2,3])
    count_arr_below = pop_in_arr_below(count_arr_ori, value[i])
    print('count_arr_below:{a}'.format(a=count_arr_below))

In [31]:
def pop_in_arr_above(count_arr_above, count_arr_above_min, value):
    """
    input:
        count_arr_above: list of integers, old above median heap
        count_arr_above_min: int, the old value represented by location [0] of the above median heap
        value: int, value to be popped into the below median heap
    output:
        count_arr_above: list of integers, new above median heap
        count_arr_above_min: int, the new value represented by location [0] of the above median heap
    """
    if count_arr_above_min <= value < count_arr_above_min + len(count_arr_above):
        min_diff = value - count_arr_above_min
        count_arr_above[min_diff] += 1
    elif value >= count_arr_above_min + len(count_arr_above):
        min_diff = value - (count_arr_above_min + len(count_arr_above) - 1)
        count_arr_above_new = [0] * (len(count_arr_above) + min_diff)
        count_arr_above_new[:len(count_arr_above)] = count_arr_above
        count_arr_above_new[len(count_arr_above_new) - 1] += 1
        count_arr_above = count_arr_above_new
    else:
        min_diff = count_arr_above_min - value
        count_arr_above_new = [0] * (min_diff + len(count_arr_above))
        count_arr_above_new[count_arr_above_min - value:] = count_arr_above
        count_arr_above = count_arr_above_new
        count_arr_above[0] += 1
        count_arr_above_min = value
    return count_arr_above, count_arr_above_min

In [34]:
# Test pop in array above
value = np.array([4,5,6,8])
for i in range(len(value)):
    count_arr_ori = np.array([1,2,3])
    count_arr_above_min_ori = 5
    count_arr_above, count_arr_above_min = pop_in_arr_above(count_arr_ori, count_arr_above_min_ori, value[i])
    print('count_arr_above:{a}, count_arr_above_min:{b}'.format(a=count_arr_above, b=count_arr_above_min))

count_arr_above:[1, 1, 2, 3], count_arr_above_min:4
count_arr_above:[2 2 3], count_arr_above_min:5
count_arr_above:[1 3 3], count_arr_above_min:5
count_arr_above:[1, 2, 3, 1], count_arr_above_min:5


In [43]:
def shrink_count_arr_below(count_arr_below):
    """
    input:
        count_arr_below: list of integers, old below median heap
    output:
        count_arr_below: list of integers, shrinked below median heap
    """
    while count_arr_below[len(count_arr_below) - 1] == 0:
        count_arr_below = count_arr_below[:len(count_arr_below) - 1]
    return count_arr_below

In [44]:
# Test shrink_count_arr_below
count_arr_below = np.array([0,2,4,5,0,6,0,0])
print('count_arr_below:{a}'.format(a=shrink_count_arr_below(count_arr_below)))

count_arr_below:[0 2 4 5 0 6]


In [45]:
def shrink_count_arr_above(count_arr_above, count_arr_above_min):
    """
    input:
        count_arr_above: list of integers, old above median heap
        count_arr_above_min:  int, the old value represented by location [0] of the above median heap
    output:
        count_arr_above: list of integers, shrinked above median heap
        count_arr_above_min:  int, the old value represented by location [0] of the above median heap
    """
    while count_arr_above[0] == 0:
        count_arr_above = count_arr_above[1:len(count_arr_above)]
        count_arr_above_min += 1
    return count_arr_above, count_arr_above_min

In [46]:
# Test shrink_count_arr_above
count_arr_above = np.array([0,0,1,3,0,1])
count_arr_above_min = 3
count_arr_above, count_arr_above_min = shrink_count_arr_above(count_arr_above, count_arr_above_min)
print('count_arr_above:{a}, count_arr_above_min:{b}'.format(a=count_arr_above, b=count_arr_above_min))

count_arr_above:[1 3 0 1], count_arr_above_min:5


In [47]:
def update_count_arr_below(trans_num, count_arr_below, run_median, trans_amt):
    """
    input:
        trans_num: int, current transaction number for the recipient-zip combination
        count_arr_below: list of integers, old below median heap
        run_median: int, old running median
        trans_amt
    output:
        count_arr_below: list of integers, new below median heap
        run_median: int, new running median
    """
    if trans_num == 2:
        count_arr_below_max = min(run_median, trans_amt)
        count_arr_below = [0] * (count_arr_below_max + 1)  # Given that round is always up,put the small array first
        count_arr_below[count_arr_below_max] += 1
        run_median = 0  # Not contributing number to be the new median
    elif trans_num % 2 == 1:
        if trans_amt > len(count_arr_below) - 1:
            run_median = 0
        elif trans_amt <= len(count_arr_below) - 1:
            run_median = len(count_arr_below) - 1  # Pop-out max value in count_arr_below as the median
            count_arr_below[trans_amt] += 1
            count_arr_below[len(count_arr_below) - 1] -= 1
            if count_arr_below[len(count_arr_below) - 1] == 0:
                count_arr_below = shrink_count_arr_below(count_arr_below)
    elif trans_num % 2 == 0:
        if trans_amt <= run_median:  # pop in trans_amt into count_arr_below, run_median will be in count_arr_above
            count_arr_below = pop_in_arr_below(count_arr_below, trans_amt)
        else:
            count_arr_below = pop_in_arr_below(count_arr_below, run_median)
        run_median = 0

    return count_arr_below, run_median

In [50]:
# Test update_count_arr_below
test_list = [[2, [], 4, 2], [3, [0,0,1], 4, 2],  [4, [0,0,1], 4, 2]]
for i in range(len(test_list)):
    trans_num, count_arr_below, run_median, trans_amt = test_list[i]
    count_arr_below, run_median = update_count_arr_below(trans_num, count_arr_below, run_median, trans_amt)
    print('count_arr_below:{a}, run_median_below:{b}'.format(a=count_arr_below, b=run_median))

count_arr_below:[0, 0, 1], run_median_above:0
count_arr_below:[0, 0, 1], run_median_above:2
count_arr_below:[0, 0, 2], run_median_above:0


In [67]:
def update_count_arr_above(trans_num, count_arr_above, count_arr_above_min, run_median, trans_amt):
    """
    input:
        trans_num: int, current transaction number for the recipient-zip combination
        count_arr_above: list of integers, old above median heap
        count_arr_above_min: int, the old value represented by location [0] of the above median heap
        run_median: int, old running median
        trans_amt: int, transaction amount from new coming entry
    output:
        count_arr_above: list of integers, new above median heap
        count_arr_above_min: int, the new value represented by location [0] of the above median heap
        run_median: int, new running median
    """
    if trans_num == 2:
        count_arr_above_min = max(run_median, trans_amt)
        count_arr_above = [0]
        count_arr_above[0] += 1
        run_median = 0  # Not contributing number to be the new median
    elif trans_num % 2 == 1:
        if trans_amt < count_arr_above_min:
            run_median = 0
        elif trans_amt >= count_arr_above_min:
            run_median = count_arr_above_min  # Pop-out min value in count_arr_above as the median
            count_arr_above, count_arr_above_min = pop_in_arr_above(count_arr_above, count_arr_above_min, trans_amt)
            count_arr_above[0] -= 1
            if count_arr_above[0] == 0:
                count_arr_above, count_arr_above_min = shrink_count_arr_above(count_arr_above, count_arr_above_min)
    elif trans_num % 2 == 0:
        if trans_amt <= run_median:  # Pop in run_median in count_arr_above
            count_arr_above, count_arr_above_min = pop_in_arr_above(count_arr_above, count_arr_above_min, run_median)
        else:
            count_arr_above, count_arr_above_min = pop_in_arr_above(count_arr_above, count_arr_above_min, trans_amt)
        run_median = 0
    return count_arr_above, count_arr_above_min, run_median

In [68]:
# Test update_count_arr_below
test_list = [[2, [], [], 4, 6], [3, [1], 5, 4, 6],  [4, [1,0,0,1], 4, 3, 6]]
for i in range(len(test_list)):
    trans_num, count_arr_above, count_arr_above_min, run_median, trans_amt = test_list[i]
    count_arr_above, count_arr_above_min, run_median = update_count_arr_above(trans_num, count_arr_above, count_arr_above_min, run_median, trans_amt)
    print('count_arr_above:{a}, count_arr_above_min:{b}, run_median_above:{c}'.format(a=count_arr_above, b=count_arr_above_min,c=run_median))

count_arr_above:[1], count_arr_above_min:6, run_median_above:0
count_arr_above:[1], count_arr_above_min:6, run_median_above:5
count_arr_above:[1, 0, 1, 1], count_arr_above_min:4, run_median_above:0


In [69]:
def running_median(trans_amt, list_zip=[]):
    """
    input:
        trans_amt: int, most recent transaction_amt entry
        list_zip: list that contains old heap information, run_median, transaction number, total amount
    output:
        list_zip: list that contains new heap information, run_median, transaction number, total amount
    """
    if len(list_zip) is 0:
        trans_num = 1
        count_arr_below = []  # Store heap as counting sort array for below median
        count_arr_above = []  # Store heap as counting sort array for above median
        count_arr_above_min = 0  # For the above heap, number that represent lowest entry
        run_median = trans_amt
        ttl_amt = trans_amt
    else:
        # Retrieve information from dictionary
        count_arr_below = list_zip[0]
        count_arr_above = list_zip[1]
        count_arr_above_min = list_zip[2]
        run_median = list_zip[3]
        trans_num = list_zip[4]
        ttl_amt = list_zip[5]

        # Update trans_num and ttl_amt
        trans_num = trans_num + 1
        ttl_amt = ttl_amt + trans_amt

        # Update count_arr_below, count_arr_above, count_arr_above_min
        count_arr_below, run_median_below = update_count_arr_below(trans_num, count_arr_below, run_median, trans_amt)
        count_arr_above, count_arr_above_min, run_median_above = update_count_arr_above(trans_num, count_arr_above,
                                                                                        count_arr_above_min, run_median,
                                                                                        trans_amt)
        count_arr_below_max = len(count_arr_below) - 1

        # Calculate run median
        if run_median_below + run_median_above == 0:
            if trans_num % 2 == 0:
                run_median = round_half((count_arr_below_max + count_arr_above_min) / 2)
            else:
                run_median = trans_amt
        elif run_median_below != 0:
            run_median = run_median_below
        else:
            run_median = run_median_above

    list_zip = [count_arr_below, count_arr_above, count_arr_above_min, run_median, trans_num, ttl_amt]
    return list_zip

# Dependency function
def round_half(x):
    """
    input:
        x: float
    output: int
    """
    if x - int(x) < 0.5:
        return int(x)
    else:
        return int(x) + 1

In [70]:
def median_zip(itcont_list):
    """
    input:
        itcont_list: list, donation information entries
    output:
        output_by_zip: list of strings: 'recipient|zip|running_median|donation_number|total_donation'
    """
    input_len = len(itcont_list)
    dict_zip = {}
    output_by_zip = []

    for i in range(input_len):  # Perform serial processing for each entry
        itcont = itcont_list[i].split('|')
        core_info = info_extract(itcont)
        if core_info is None:   # Check if the entry contains core information
            continue

        # Check if ZIP and tranaction amount is valid
        try:
            len_check = core_info['ZIP_CODE'][4]  # check if 'ZIP_CODE' has at least 5 characters
            zip_code = int(core_info['ZIP_CODE'])  # check if 'ZIP_CODE' contains only digit
            trans_amt = int(core_info['TRANSACTION_AMT'])  # check if 'TRANSACTION_AMT' contains only digit
        except:
            continue

        # Create one dictionary for each recipient and zip
        key = core_info['CMTE_ID'] + core_info['ZIP_CODE']
        # Update the dictionary with new transaction amount
        if key not in dict_zip:       # First entry, initiate the list
            list_zip = []
        else:
            list_zip = dict_zip[key]

        # Obtain the running median given the current information and new transaction_amount
        dict_zip[key] = running_median(trans_amt, list_zip)
        run_median, trans_num, ttl_amt = dict_zip[key][3:6]

        output_by_zip.append(
            '|'.join([core_info['CMTE_ID'], core_info['ZIP_CODE'], str(run_median), str(trans_num), str(ttl_amt)]))
    return output_by_zip

In [72]:
output = median_zip(itcont_list)
output

['C00177436|30004|384|1|384',
 'C00384818|02895|250|1|250',
 'C00177436|30750|230|1|230',
 'C00177436|04105|384|1|384',
 'C00384818|02895|292|2|583',
 'C00177436|04105|384|2|768']

In [73]:
def date_format(date_str, amt_str):
    """
    input:
        date_str: string
        amt_str: string
    output:
        True/False
    """
    try:
        month = int(date_str[:2])
        date = int(date_str[2:4])
        yr = int(date_str[-4:])
        amt = int(amt_str)

        if (0 < month < 13) & (0 < date < 32) & (1000 < yr < 2018):
            return True
        else:
            return False
    except:
        return False

In [74]:
# Define the function to calculate the median by recipient and date
# Find unique combination of CMTE_ID and date

# Combine all the strings
def median_date(itcont_list):
    """
    Calculate median and total amount of donation based on recipient-date info
    --------------
     input:
        itcont_list: list, donation information entries
    output:
        output_by_zip: list of strings: 'recipient|date|median|donation_number|total_donation'
    """
    # Batch processing with all the entries
    itcont = '|'.join(itcont_list).split('|')

    # Obtain the length with valid data info
    num_entry = int(len(itcont) / 21)

    # Find the entries with Other_Id as [] and TRANS_AMT, TRANS_DT of correct format:
    key_id = [i for i in range(num_entry) if
              (itcont[i * 21 + 15] == '') & (date_format(itcont[i * 21 + 13], itcont[i * 21 + 14])) & (
              len(itcont[i * 21]) > 0)]

    # Generate combination keys with CMTE_ID and TRANS_DT
    keys = [itcont[i * 21] + itcont[i * 21 + 13] for i in key_id]
    unique_key = list(set(keys))
    # Sort the key to prepare for output order
    unique_key.sort()
    output_by_date = []

    for j in range(len(unique_key)):
        # Find all the entries that correspond to the same recipient-date key
        indices = [key_id[i] for i, x in enumerate(keys) if x == unique_key[j]]
        trans_num = len(indices)
        trans_amt = [int(itcont[i * 21 + 14]) for i in indices]
        ttl_trans = sum(trans_amt)
        if len(trans_amt) == 1:
            median_trans = round(trans_amt[0])
        elif len(trans_amt) == 2:
            median_trans = round(ttl_trans / 2)
        elif len(trans_amt) % 2 == 0:
            trans_amt.sort()
            mid_idx = int(len(trans_amt) / 2 - 1)
            median_trans = round(sum(trans_amt[mid_idx:mid_idx + 2]) / 2)
        else:
            trans_amt.sort()
            mid_idx = int(len(trans_amt) / 2)
            median_trans = round(trans_amt[mid_idx])
        len_key = len(unique_key[j])

        output_by_date.append('|'.join(
            [unique_key[j][:len_key - 8], unique_key[j][len_key - 8:], str(median_trans), str(trans_num),
             str(ttl_trans)]))

    return output_by_date

In [75]:
output_by_date = median_date(itcont_list)
output_by_date

['C00177436|01312017|384|4|1382', 'C00384818|01122017|292|2|583']